In [1]:
from evaluate import load
from evaluate.module import EvaluationModule
from datasets import load_dataset, Dataset
from models.qg import QG
import torch

qg = QG("the-coorporation/t5-small-qg-2.0", "the-coorporation/t5-small-qg-2.0")

/Users/philiphyltoft/miniconda3/envs/hg_training/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## BLEU

In [2]:
bleu: EvaluationModule = load("bleu")

## METEOR

In [3]:
meteor: EvaluationModule = load("meteor")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/philiphyltoft/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/philiphyltoft/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/philiphyltoft/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
squad: Dataset = load_dataset("the-coorporation/the_squad_qg")
validation = squad['validation']
validation

No config specified, defaulting to: the_squad_qg/v2
Found cached dataset the_squad_qg (/Users/philiphyltoft/.cache/huggingface/datasets/the-coorporation___the_squad_qg/v2/2.0.0/cd5e3386e9e3124c24114539fb5124093fa58d99c1a7292451b8801a94a2aca5)
100%|██████████| 2/2 [00:00<00:00, 137.66it/s]


Dataset({
    features: ['context', 'questions'],
    num_rows: 1204
})

In [20]:
references: list[dict] = [entry['questions'].replace(" {sep_token}", "") for entry in validation]

predictions: list[dict] = []

device = "cuda" if torch.cuda.is_available() else "cpu"

generator_args = {
    "max_length": 512,
    "num_beams": 4,
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
}

for i, entry in enumerate(validation):
    # if i == 1000:
    #     break
    
    context: str = entry['context']
    input_string = "generate questions: " + context + " </s>"

    input_ids = qg._tokenizer.encode(input_string, return_tensors="pt").to(device)
    result = qg._model.generate(input_ids, **generator_args).to(device)
    questions = qg._tokenizer.decode(result[0], skip_special_tokens=True)
    questions = questions.replace("<sep>", "")
    predictions.append(questions)

In [17]:
# print(len(references[:5]))
# print(len(predictions))

print(references[0])
print(predictions[0])

Geographically speaking, where is California's north - south midway point in terms of latitude? How many miles south of San Jose is the north - south midway point located? The term "southern" California usually refers to how many of the southern-most counties of the state? Other than Point Conception, what landmark is used in the other definition of southern California? Point Conception is an example of a landmark among what boundary of southern California?
What is Southern California not a formal geographic designation? What is the north-south midway point of California? When California is divided into two areas, the term "southern California" usually refers to what?


In [16]:
bleu.compute(references=references[:5], predictions=predictions)

{'bleu': 0.1590160494240242,
 'precisions': [0.48936170212765956,
  0.21660649819494585,
  0.125,
  0.0749063670411985],
 'brevity_penalty': 0.8958976718854088,
 'length_ratio': 0.9009584664536742,
 'translation_length': 282,
 'reference_length': 313}

In [ ]:
meteor.compute(references=references[:5], predictions=predictions)